In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from matplotlib import colors
from PIL import Image
import napari
import skimage.io as io
import seaborn as sns
import pandas as pd
import colorcet as cc
import sys
import tifffile
from scipy import ndimage
from skimage.segmentation import find_boundaries

sys.path.append('..')
from spatialMI_functions import plot_layers_napari, crop_black_margins, plot_pixie_maps

import warnings
warnings.filterwarnings("ignore")


/Users/florian_wuennemann/miniconda3/envs/spatialomics_MI/lib/python3.11/site-packages/geopandas/_compat.py:124: UserWarning: The Shapely GEOS version (3.11.2-CAPI-1.17.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


In [2]:
# Explore some different color options

## Lunaphore images
channel_order = ["DAPI",
                 "TNNT2","ANKRD1","PDGFRa","CD31","aSMA",
                 "CD45","CCR2","TREM2","CD68","MPO"]
colorblind = sns.color_palette("colorblind")
glasbey = sns.color_palette(cc.glasbey)
comet_images_color = ['cyan',
                      glasbey[10],glasbey[7],glasbey[6],colorblind[3],glasbey[0],
                      glasbey[1],glasbey[2],glasbey[4],glasbey[12],"magenta"]

## Pixel maps
pixel_maps_color = sns.color_palette(cc.glasbey)

## Cell clusters
cell_cluster_color = sns.color_palette(cc.glasbey)

# Visualize the colors in the color palette
pixel_maps_color

[(0.8392156862745098, 0.0, 0.0),
 (0.5490196078431373, 0.23137254901960785, 1.0),
 (0.00392156862745098, 0.5294117647058824, 0.0),
 (0.0, 0.6745098039215687, 0.7764705882352941),
 (0.592156862745098, 1.0, 0.0),
 (1.0, 0.49411764705882355, 0.8196078431372549),
 (0.4196078431372549, 0.0, 0.30980392156862746),
 (1.0, 0.6470588235294118, 0.1843137254901961),
 (0.3411764705882353, 0.23137254901960785, 0.0),
 (0.0, 0.33725490196078434, 0.34901960784313724),
 (0.0, 0.0, 0.8666666666666667),
 (0.0, 0.9921568627450981, 0.8117647058823529),
 (0.6313725490196078, 0.4588235294117647, 0.4117647058823529),
 (0.7372549019607844, 0.7137254901960784, 1.0),
 (0.5843137254901961, 0.7098039215686275, 0.4666666666666667),
 (0.7490196078431373, 0.011764705882352941, 0.7215686274509804),
 (0.39215686274509803, 0.32941176470588235, 0.4549019607843137),
 (0.4745098039215686, 0.0, 0.0),
 (0.027450980392156862, 0.4549019607843137, 0.8470588235294118),
 (0.9921568627450981, 0.9568627450980393, 0.5647058823529412),
 (0.0, 0.29411764705882354, 0.0),
 (0.5568627450980392, 0.4745098039215686, 0.0),
 (1.0, 0.4470588235294118, 0.4),
 (0.9294117647058824, 0.7215686274509804, 0.7215686274509804),
 (0.36470588235294116, 0.49411764705882355, 0.4),
 (0.6039215686274509, 0.8941176470588236, 1.0),
 (0.9215686274509803, 0.0, 0.4666666666666667),
 (0.6470588235294118, 0.4823529411764706, 0.7215686274509804),
 (0.34901960784313724, 0.0, 0.6392156862745098),
 (0.011764705882352941, 0.7764705882352941, 0.0),
 (0.6196078431372549, 0.29411764705882354, 0.0),
 (0.611764705882353, 0.23137254901960785, 0.30980392156862746),
 (0.792156862745098, 0.7647058823529411, 0.0),
 (0.4392156862745098, 0.5098039215686274, 0.592156862745098),
 (0.0, 0.6862745098039216, 0.5372549019607843),
 (0.5098039215686274, 0.5294117647058824, 1.0),
 (0.36470588235294116, 0.21176470588235294, 0.23137254901960785),
 (0.2196078431372549, 0.0, 0.0),
 (0.9921568627450981, 0.7490196078431373, 1.0),
 (0.7411764705882353, 0.9019607843137255, 0.7490196078431373),
 (0.8588235294117647, 0.42745098039215684, 0.00392156862745098),
 (0.5764705882352941, 0.7215686274509804, 0.7098039215686275),
 (0.8941176470588236, 0.3215686274509804, 1.0),
 (0.1843137254901961, 0.3215686274509804, 0.5098039215686274),
 (0.7647058823529411, 0.4, 0.5647058823529412),
 (0.32941176470588235, 0.3843137254901961, 0.12156862745098039),
 (0.7686274509803922, 0.6196078431372549, 0.4470588235294118),
 (0.011764705882352941, 0.5098039215686274, 0.5294117647058824),
 (0.4117647058823529, 0.9019607843137255, 0.5019607843137255),
 (0.5019607843137255, 0.14901960784313725, 0.5647058823529412),
 (0.42745098039215684, 0.7019607843137254, 1.0),
 (0.30196078431372547, 0.2, 1.0),
 (0.5215686274509804, 0.6392156862745098, 0.00392156862745098),
 (0.9921568627450981, 0.011764705882352941, 0.792156862745098),
 (0.7568627450980392, 0.6470588235294118, 0.7686274509803922),
 (0.7686274509803922, 0.33725490196078434, 0.27450980392156865),
 (0.4588235294117647, 0.3411764705882353, 0.23921568627450981),
 (0.00392156862745098, 0.403921568627451, 0.25882352941176473),
 (0.0, 0.8392156862745098, 0.8313725490196079),
 (0.8549019607843137, 0.8745098039215686, 1.0),
 (0.9764705882352941, 1.0, 0.0),
 (0.4117647058823529, 0.403921568627451, 0.6862745098039216),
 (0.7647058823529411, 0.592156862745098, 0.0),
 (0.8823529411764706, 0.803921568627451, 0.611764705882353),
 (0.8549019607843137, 0.5843137254901961, 1.0),
 (0.7294117647058823, 0.011764705882352941, 0.9921568627450981),
 (0.5686274509803921, 0.3215686274509804, 0.5098039215686274),
 (0.6274509803921569, 0.0, 0.4470588235294118),
 (0.33725490196078434, 0.6039215686274509, 0.32941176470588235),
 (0.8274509803921568, 0.5490196078431373, 0.5568627450980392),
 (0.21176470588235294, 0.26666666666666666, 0.14901960784313725),
 (0.592156862745098, 0.6470588235294118, 0.7647058823529411),
 (0.5568627450980392, 0.5490196078431373, 0.3686274509803922),
 (1.0, 0.27450980392156865, 0.0),
 (0.7843137254901961, 1.0, 0.97647

In [10]:
# Assign each individual label in new_mask the corresponding color value from the color table
pixel_dir = '../../../results/seqIF/pixie_pixel_masks_0.05'

## Get representative pixel categories from one sample
sample_folder = pixel_dir + "/" + "Control_14" + "/"
pixel_categories = pd.read_csv(sample_folder + "population_pixel_mask.csv")

## Assign color palettes to the different pixel categories
## Pixel maps
pixel_maps_color =sns.color_palette(cc.glasbey) 
color_palette_use = pixel_maps_color[1:pixel_categories.shape[0]+1]
color_dict = {0: 'white'}
for i,region_id in enumerate(pixel_categories['region_id']):
    if pixel_categories['region_name'][i] == "background":
        color_dict[region_id] = "white"
    else:
        color_dict[region_id] = color_palette_use[i]
color_dict

{0: 'white',
 1: (0.5490196078431373, 0.23137254901960785, 1.0),
 2: (0.00392156862745098, 0.5294117647058824, 0.0),
 5: (0.0, 0.6745098039215687, 0.7764705882352941),
 8: (0.592156862745098, 1.0, 0.0),
 11: (1.0, 0.49411764705882355, 0.8196078431372549),
 12: (0.4196078431372549, 0.0, 0.30980392156862746),
 17: (1.0, 0.6470588235294118, 0.1843137254901961),
 18: (0.3411764705882353, 0.23137254901960785, 0.0),
 20: (0.0, 0.33725490196078434, 0.34901960784313724),
 24: (0.0, 0.0, 0.8666666666666667),
 30: 'white',
 31: (0.6313725490196078, 0.4588235294117647, 0.4117647058823529)}

In [12]:
# Translate color_dict color values from rgb to hex
color_dict_hex = {}
for key in color_dict:
    color_dict_hex[key] = colors.to_hex(color_dict[key])
color_dict_hex

{0: '#ffffff',
 1: '#8c3bff',
 2: '#018700',
 5: '#00acc6',
 8: '#97ff00',
 11: '#ff7ed1',
 12: '#6b004f',
 17: '#ffa52f',
 18: '#573b00',
 20: '#005659',
 24: '#0000dd',
 30: '#ffffff',
 31: '#a17569',
 None: '#000000'}

: 

In [11]:
# Assign each individual label in new_mask the corresponding color value from the color table
pixel_dir = '../../../results/seqIF/pixie_pixel_masks_0.05'
outdir = "../../output/seqIF/pixie_maps/"
# Generate an image of the pixel map from pixie for each sample. These will be put together in Illustrator afterward
for sample in os.listdir(pixel_dir):
    if os.path.isdir(os.path.join(pixel_dir, sample)):
        print(sample)
        sample_dir= pixel_dir + "/" + sample + "/"
        pixel_img = tifffile.TiffFile(sample_dir + "population_pixel_mask.tiff").asarray()

        # Read in a csv file with the ROI coordinates
        sample_roi = pd.read_csv("/Users/florian_wuennemann/1_Projects/MI_project/mi_spatialomics/annotations/SeqIF/heart_region_rois/"+ sample +"_heart.csv")

        # use the bounding box positions in the roi to subset the pixel_img and the mask
        y_min = int(sample_roi['axis-0'].min())
        y_max = int(sample_roi['axis-0'].max())
        x_min = int(sample_roi['axis-1'].min())
        x_max = int(sample_roi['axis-1'].max())

        # crop the pixel_img and the mask
        pixel_img_cropped = pixel_img[y_min:y_max, x_min:x_max]

        viewer = napari.Viewer()
        viewer.add_labels(pixel_img_cropped, visible=True, name=sample, opacity = 1, color = color_dict)
        full_screenshot = outdir+sample+".pixel_map.full_image.wborder.png"
        viewer.screenshot(path=full_screenshot, scale=3)
        crop_screenshot = outdir+sample+".pixel_map.full_image.clean.png"
        crop_black_margins(full_screenshot, crop_screenshot)

Control_14
Control_13
Control_12
4h_97
24h_83
4h_96
24h_86
48h_79


Context leak detected, msgtracer returned -1


48h_76


In [4]:
## Manual exploration of pixel maps in  Napari
viewer = napari.Viewer()
for folder in os.listdir(pixel_dir):
    if os.path.isdir(os.path.join(pixel_dir, folder)) and folder != "napari_screenshots":
        print(folder)
        # Do something with the folder here
        sample_folder = pixel_dir + "/" + folder + "/"
        pixel_img = tifffile.TiffFile(sample_folder + "population_pixel_mask.tiff").asarray()
        viewer.add_labels(pixel_img, visible=False, name=folder, opacity = 1, color = color_dict)

28-Feb-24 16:46:31 - bfio.backends - WARNING  - Java backend is not available. This could be due to a missing dependency (jpype).


Control_14
Control_13
Control_12
4h_97
24h_83
4h_96
24h_86
48h_79
48h_76


In [ ]:
# Test code for a single sample
sample = "4h_96"
folder = sample

sample_folder = pixel_dir + "/" + folder + "/"
pixel_img = tifffile.TiffFile(sample_folder + "population_pixel_mask.tiff").asarray()


# Read in a csv file with the ROI coordinates
sample_roi = pd.read_csv("/Users/florian_wuennemann/1_Projects/MI_project/mi_spatialomics/annotations/SeqIF/heart_region_rois/"+ sample +"_heart.csv")

# use the bounding box positions in the roi to subset the pixel_img and the mask
x_min = int(sample_roi['axis-0'].min())
x_max = int(sample_roi['axis-0'].max())
y_min = int(sample_roi['axis-1'].min())
y_max = int(sample_roi['axis-1'].max())

# crop the pixel_img and the mask
pixel_img_cropped = pixel_img[y_min:y_max, x_min:x_max]


viewer = napari.Viewer()
viewer.add_labels(pixel_img_cropped, visible=True, name=folder, opacity = 1, color = color_dict)

full_screenshot = outdir+sample+".pixel_map.full_image.wborder.png"
viewer.screenshot(path=full_screenshot, scale=3)

crop_screenshot = outdir+sample+".pixel_map.full_image.clean.png"
crop_black_margins(full_screenshot, crop_screenshot)